# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile
import btrack
print(btrack.__version__)
import os

0.6.1.dev30


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [4]:
row = 3
column = 5

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
#                              set_channel=1,
                             set_time = 0,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

In [20]:
%%time
images = images.compute().compute()

CPU times: user 2h 52min 4s, sys: 17min 32s, total: 3h 9min 36s
Wall time: 7min 29s


In [6]:
images.shape

(1, 2, 6048, 6048)

# Load tracks

In [7]:
tracks_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/{row, column}.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    tracks = hdf.tracks
    segmentation = hdf.segmentation
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)

[INFO][2023/05/25 11:55:59 AM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/05/25 11:55:59 AM] Loading tracks/obj_type_1
[INFO][2023/05/25 11:55:59 AM] Loading LBEP/obj_type_1
[INFO][2023/05/25 11:55:59 AM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/05/25 11:56:11 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/05/25 11:56:11 AM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5


In [9]:
gt_dict 

{425: True,
 694: True,
 403: False,
 427: True,
 453: False,
 217: True,
 1002: False,
 1893: False,
 1986: False,
 2192: False,
 401: True,
 412: False,
 2499: False,
 1446: False,
 1003: True,
 790: False,
 1552: False,
 2862: False,
 432: True,
 456: True,
 461: True,
 416: True,
 1892: False,
 430: True,
 1226: True,
 1783: True,
 '429,2818': 'Link',
 431: True,
 407: True,
 '2675,404': 'Link',
 398: True,
 423: True,
 424: True,
 428: False,
 418: False,
 411: False,
 '411,1683': 'Link',
 1058: False,
 466: True,
 465: True,
 397: True,
 426: True,
 419: True,
 1735: True,
 420: True,
 421: True,
 802: True,
 455: False,
 406: False,
 410: True,
 '411,1118': 'Link',
 1118: True,
 '1114,2130': 'Link',
 414: True,
 459: True,
 2234: True,
 470: True,
 437: True,
 '1649,2453': 'Link',
 '2453,3126': 'Link',
 444: True,
 865: False,
 441: True,
 318: False,
 2952: False,
 1596: False,
 1650: False,
 442: False,
 339: True,
 '317,1432': 'Link',
 '1432,2223': 'Link',
 325: True,
 335: T

In [14]:
filtered_tracks = [t for t in tracks if t.ID in gt_dict.keys() and gt_dict[t.ID] == True]

In [15]:
len(filtered_tracks)

236

In [16]:
napari_filtered_tracks, properties, graph = btrack.utils.tracks_to_napari(filtered_tracks, 
                                                                    ndim=2)

### Recolour tracks

In [17]:
col_segmentation = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

# Launch napari image viewer

In [15]:
images.shape

(75, 2, 1, 6048, 6048)

In [20]:
images[0,:,0:4200,:]

dask.array<getitem, shape=(2, 4200, 6048), dtype=uint8, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

In [24]:
viewer = napari.Viewer()

# viewer.add_image(images, 
#                  channel_axis=1,
#                  name=["macrophage", "mtb"],
#                  colormap=["green",  "magenta"],
# #                  contrast_limits=[[100, 6000], [100, 2000]],
#                  contrast_limits=[[0,450], [0,450]], 
#                  visible = True
#                  )
viewer.add_image(images[0,:,0:4200,:], 
                 channel_axis=0,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
#                  contrast_limits=[[100, 6000], [100, 2000]],
                 contrast_limits=[[0,450], [0,450]], 
                 visible = True
                 )
# viewer.add_labels(segmentation, 
# #                   num_colors= 1,
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                   name = 'segmentation'
#                  )
# viewer.add_labels(col_segmentation, 
#                   name = 'recolored segmentation'
#                   #scale=(10, 1, 1,), 
#                   #color='yellow'
#                  )
# viewer.add_tracks(napari_tracks, scale = (1,5.04,5.04)
# #                     properties=properties, 
# #                     graph=graph, 
# #                     name="Properly downscaled tracks", 
# #                     blending="translucent",
# #                     visible=True,
# # #                     scale = (100,1,1)
#                  )
viewer.add_tracks(napari_filtered_tracks, scale = (45,5.04,5.04),
                    properties=properties, 
                    graph=graph, 
#                     name="Properly downscaled tracks", 
#                     blending="translucent",
#                     visible=True,
# #                     scale = (100,1,1)
                 )

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Tracks layer 'napari_filtered_tracks' at 0x7ff4c357aa90>

In [25]:
viewer.layers['napari_filtered_tracks'].scale = (30, 5.04, 5.04)

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 227, in <lambda>
    button.clicked.connect(lambda: self.trigger(name))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/action_manager.py", line 427, in trigger
    return self._actions[name].injected()
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/in_n_out/_store.py", line 773, in _exec
    result = func(**{**_kwargs, **bound.arguments})
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/image/_image_key_bindings.py", line 33, in orient_plane_normal_along_x
    orient_plane_normal_around_cursor(layer, plane_normal=(0, 0, 1))
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/layers/utils/interactivity_utils.py", line 126, in orient_plane_normal_around_cursor
    view_direction = layer._world_to_displayed_data_ray(
  File "/home/dayn/minic

In [32]:
viewer.layers['napari_filtered_tracks'].axis_labels=["time", "y", "x"]

In [29]:
viewer.add_tracks(ds_napari_tracks,
#                     properties=properties, 
#                     graph=graph, 
                    name="hacky downscaled tracks", 
                    blending="translucent",
                    visible=True,
#                     scale = (1,1,1)
                )
# viewer.add_labels(ds_segmentation, 
#                  name = 'downscaled segmentation')

viewer.add_labels(ds_col_segmentation, 
                  name = 'downscaled coloured segmentation',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

viewer.add_labels(ds_col_segmentation_new_tracks, 
                  name = 'downscaled coloured segmentation new tracks',
                  scale=(1, 1, 1,), 
#                   color={1:'yellow'}
                 )

<Labels layer 'downscaled coloured segmentation new tracks' at 0x7f15c273ca30>